<a href="https://colab.research.google.com/github/HunterWelsch/BigData-AWS-RDS/blob/master/Challenge_16_e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

In [0]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-02-20 19:04:47--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-02-20 19:04:47 (8.63 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigDataHW").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
!pip install pyspark

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
e_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), sep="\t", header=True)
e_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [0]:
from pyspark.sql.types import IntegerType, DateType
e_data_df = e_data_df.withColumn("customer_id", e_data_df["customer_id"].cast(IntegerType()))
e_data_df = e_data_df.withColumn("product_parent", e_data_df["product_parent"].cast(IntegerType()))
e_data_df = e_data_df.withColumn("review_date", e_data_df["review_date"].cast(DateType()))
e_data_df = e_data_df.withColumn("star_rating", e_data_df["star_rating"].cast(IntegerType()))
e_data_df = e_data_df.withColumn("helpful_votes", e_data_df["helpful_votes"].cast(IntegerType()))
e_data_df = e_data_df.withColumn("total_votes", e_data_df["total_votes"].cast(IntegerType()))

In [0]:
customers = e_data_df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|             6|
|   28871066|             1|
|    6711334|             2|
|   46393060|             1|
|   48101124|             1|
|   38209321|             1|
|   47108763|             4|
|   43252205|             1|
|   41283385|             1|
|   46909180|             5|
|    5453476|             2|
|   12406466|             3|
|   48146680|             1|
|    5953456|             1|
|   16457323|             5|
|   24540309|             1|
|   43366878|             2|
|   28796717|             1|
|   28258386|             1|
|   26079415|             2|
+-----------+--------------+
only showing top 20 rows



In [0]:
review_id_table = e_data_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")

In [0]:
products_table = e_data_df.select("product_id", "product_title")

In [0]:
vine_table = e_data_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine")
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
|R1DGA6UQIVLKZ7|          5|            0|          0|   N|
| RLQT3V8SMNIBH|          5|            0|          0|   N|
|R3T9GZS2TMXZGM|          1|            0|          0|   N|
|R24HVAEYP5PLDN|          5|            

In [0]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cukxpdx4oe4k.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres",
          "password": "Umiami#11",
          "driver":"org.postgresql.Driver"}

In [0]:
# Write DataFrame to active_user table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [0]:
# Write DataFrame to active_user table in RDS
products_table.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Write DataFrame to active_user table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)